In [1]:
# 원리 이해 

weights = [0.55,0.45] # initialize the weights
biases = [0.15] # initialize the biases

# set input 
x_1 = 0.5 # input 1
x_2 = 0.35 # input 2

# compute weighted sum of the inputs , 1st hidden layer 
z_11 = x_1 * weights[0] + x_2 * weights[1] + biases[0]
print('The weighted sum of the inputs at the first node in the hidden layer is {}'.format(z_11))

# use sigmoid activation function 
a_11 = 1.0 / (1.0 + np.exp(-z_11))


print(a_11)


The weighted sum of the inputs at the first node in the hidden layer is 0.5825
0.6416424532015634


In [ ]:
# 초기화 (인풋, 히든레이어 갯수, 히든 노드 갯수, 아웃풋 노드 갯수)
def initialize_network(num_inputs, num_hidden_layers, num_nodes_hidden, num_nodes_output):
    
    num_nodes_previous = num_inputs # number of nodes in the previous layer # 직전 노드 갯수 = 시작을 인풋 갯수로 한다. 

    network = {} # 전체 네트워크(레이어와 노드 정보)를 딕셔너리 형태로 저장하기 위해 초기화합니다.
    
    # loop through each layer and randomly initialize the weights and biases associated with each layer # 모든 레이어를 초기화 하는데 이때 웨이트랑 바이아스 모두 초기화 한다. 
    for layer in range(num_hidden_layers + 1): # 히든 레이어를 하나하나 돈다. 
        
        if layer == num_hidden_layers: # 만약에 히든레이어까지 다 돌고나면 
            layer_name = 'output' # name last layer in the network output # 해당 레이어는 output로 한다. 마지막 레이어니까. 
            num_nodes = num_nodes_output # 그 마지막 레이어의 노드는 num_nodes_output으로 바꿔준다. 
        else:
            layer_name = 'layer_{}'.format(layer + 1) # otherwise give the layer a number # 예시 layer = 0 이면, layer_1로 레이어 이름을 문자열화 해준다. 
            num_nodes = num_nodes_hidden[layer] 
        
        # initialize weights and bias for each node # 각 노드는 “이전 레이어에서 오는 입력”마다 가중치(여러 개)를 가지며, 추가로 편향(bias)도 하나 가집니다.
        network[layer_name] = {} # network["layer_1"] = {}
        for node in range(num_nodes): # 전체 노드에 대해서 
            node_name = 'node_{}'.format(node+1) # 각 노드에 레이블 달아준다. 
            network[layer_name][node_name] = { # network가 2중 딕셔너리 구조가 된다. 예시 network["layer_1"]["node_1"]
                'weights': np.around(np.random.uniform(size=num_nodes_previous), decimals=2),
                'bias': np.around(np.random.uniform(size=1), decimals=2),
            }
    
        num_nodes_previous = num_nodes # 레이어 생성 후, 다음 레이어가 몇 개 가중치를 가져야 할지 알려주기 위해 현재 레이어 노드 수를 ‘이전 레이어 노드 수’로 갱신합니다.

    return network # return the network


# Define the small network
small_network = initialize_network(num_inputs=5, num_hidden_layers=1, num_nodes_hidden=[3], num_nodes_output=1)


In [ ]:
def compute_weighted_sum(inputs, weights, bias): # weighted sum을 해줘야 다음 레이어로 넘어간다. 
    return np.sum(inputs * weights) + bias

from random import seed
import numpy as np

np.random.seed(12)
inputs = np.around(np.random.uniform(size=5), decimals=2)

print('The inputs to the network are {}'.format(inputs))

node_weights = small_network['layer_1']['node_1']['weights']
node_bias = small_network['layer_1']['node_1']['bias']
weighted_sum = compute_weighted_sum(inputs, node_weights, node_bias) 
print('The weighted sum at the first node in the hidden layer is {}'.format(np.around(weighted_sum[0], decimals=4)))

In [ ]:
def node_activation(weighted_sum):
    return 1.0 / (1.0 + np.exp(-1 * weighted_sum))

node_output  = node_activation(compute_weighted_sum(inputs, node_weights, node_bias))
print('The output of the first node in the hidden layer is {}'.format(np.around(node_output[0], decimals=4)))

In [ ]:
# Forward propagation = (Initialization) + (Weighted Sum) + (Node Activation) for every node in every layer, sequentially.


def forward_propagate(network, inputs): 
    
    layer_inputs = list(inputs) # start with the input layer as the input to the first hidden layer
    
    for layer in network:
        
        layer_data = network[layer] # 예: network['layer_1'] → 'layer_1'이 layer이고, 레이어 정보가 layer_data에 저장됩니다.
        
        layer_outputs = [] # 한 레이어에는 여러 노드가 있으며, 각 노드의 출력값을 저장해야 하기 때문입니다.
        for layer_node in layer_data: 

# layer_data = {
#     'node_1': {'weights': [...], 'bias': ...},
#     'node_2': {'weights': [...], 'bias': ...}
# }
        
            node_data = layer_data[layer_node] # node_data = layer_data['node_1']
        
            # compute the weighted sum and the output of each node at the same time 
            node_output = node_activation(compute_weighted_sum(layer_inputs, node_data['weights'], node_data['bias']))
            layer_outputs.append(np.around(node_output[0], decimals=4))
            
        if layer != 'output':
            print('The outputs of the nodes in hidden layer number {} is {}'.format(layer.split('_')[1], layer_outputs)) 
    
        layer_inputs = layer_outputs # set the output of this layer to be the input to next layer

    network_predictions = layer_outputs
    return network_predictions

predictions = forward_propagate(small_network, inputs)
print('The predicted value by the network for the given input is {}'.format(np.around(predictions[0], decimals=4)))

In [ ]:
# 전체 코드 
def initialize_network(num_inputs, num_hidden_layers, num_nodes_hidden, num_nodes_output):
    
    num_nodes_previous = num_inputs # number of nodes in the previous layer

    network = {}
    
    # loop through each layer and randomly initialize the weights and biases associated with each layer
    for layer in range(num_hidden_layers + 1):
        
        if layer == num_hidden_layers:
            layer_name = 'output' # name last layer in the network output
            num_nodes = num_nodes_output
        else:
            layer_name = 'layer_{}'.format(layer + 1) # otherwise give the layer a number
            num_nodes = num_nodes_hidden[layer] 
        
        # initialize weights and bias for each node
        network[layer_name] = {}
        for node in range(num_nodes):
            node_name = 'node_{}'.format(node+1)
            network[layer_name][node_name] = {
                'weights': np.around(np.random.uniform(size=num_nodes_previous), decimals=2),
                'bias': np.around(np.random.uniform(size=1), decimals=2),
            }
    
        num_nodes_previous = num_nodes

    return network # return the network

def compute_weighted_sum(inputs, weights, bias):
    return np.sum(inputs * weights) + bias

from random import seed
import numpy as np

np.random.seed(12)
inputs = np.around(np.random.uniform(size=5), decimals=2)

print('The inputs to the network are {}'.format(inputs))

node_weights = small_network['layer_1']['node_1']['weights']
node_bias = small_network['layer_1']['node_1']['bias']
weighted_sum = compute_weighted_sum(inputs, node_weights, node_bias) # 여기서 input은 뭐가 됨? 
print('The weighted sum at the first node in the hidden layer is {}'.format(np.around(weighted_sum[0], decimals=4)))

def node_activation(weighted_sum):
    return 1.0 / (1.0 + np.exp(-1 * weighted_sum))

node_output  = node_activation(compute_weighted_sum(inputs, node_weights, node_bias))
print('The output of the first node in the hidden layer is {}'.format(np.around(node_output[0], decimals=4)))



# The way we are going to accomplish this is through the following procedure:

# Start with the input layer as the input to the first hidden layer.
# Compute the weighted sum at the nodes of the current layer.
# Compute the output of the nodes of the current layer.
# Set the output of the current layer to be the input to the next layer.
# Move to the next layer in the network.
# Repeat steps 2 - 4 until we compute the output of the output layer.

def forward_propagate(network, inputs):
    
    layer_inputs = list(inputs) # start with the input layer as the input to the first hidden layer
    
    for layer in network:
        
        layer_data = network[layer]
        
        layer_outputs = [] 
        for layer_node in layer_data:
        
            node_data = layer_data[layer_node]
        
            # compute the weighted sum and the output of each node at the same time 
            node_output = node_activation(compute_weighted_sum(layer_inputs, node_data['weights'], node_data['bias']))
            layer_outputs.append(np.around(node_output[0], decimals=4))
            
        if layer != 'output':
            print('The outputs of the nodes in hidden layer number {} is {}'.format(layer.split('_')[1], layer_outputs))
    
        layer_inputs = layer_outputs # set the output of this layer to be the input to next layer

    network_predictions = layer_outputs
    return network_predictions

predictions = forward_propagate(small_network, inputs)
print('The predicted value by the network for the given input is {}'.format(np.around(predictions[0], decimals=4)))